In [1]:
import os
import re
from tqdm import tqdm

from pdf2image import convert_from_path
import pytesseract

import pypandoc

In [2]:
# Download a sample PDF file
! wget http://www.africau.edu/images/default/sample.pdf

--2021-05-06 12:39:09--  http://www.africau.edu/images/default/sample.pdf
Resolving www.africau.edu (www.africau.edu)... 104.21.63.175, 172.67.148.215
Connecting to www.africau.edu (www.africau.edu)|104.21.63.175|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3028 (3,0K) [application/pdf]
Saving to: ‘sample.pdf’

sample.pdf          100%[===================>]   2,96K  --.-KB/s    in 0s      

2021-05-06 12:39:09 (46,6 MB/s) - ‘sample.pdf’ saved [3028/3028]



In [3]:
src_file = "sample.pdf"

In [4]:
file_path, file_name = os.path.split(src_file)
file_base_name, _ = os.path.splitext(file_name)

txt_file = os.path.join(file_path, ".".join((file_base_name, "txt")))
epub_file = os.path.join(file_path, ".".join((file_base_name, "epub")))

In [5]:
# PDF file --> PIL Images

images = convert_from_path(src_file, fmt="jpeg")

In [6]:
# PIL Images --> Txt file

with open(txt_file, "w") as f:

    for image in tqdm(images):

        text = pytesseract.image_to_string(image)

        text = re.sub(
            "(?<![\r\n])(\r?\n|\r)(?![\r\n])", " ", text
        )  # Repair sentences that have \n in the middle
        text = "\n".join(
            " ".join(line.split()) for line in text.split("\n")
        )  # Remove extra whitespaces (pypandoc cannot convert them)

        f.write(f"{text}\n")

100%|██████████| 2/2 [00:01<00:00,  1.06it/s]


In [7]:
# Txt file --> Epub file

pypandoc.convert_file(txt_file, format="markdown", to="epub", outputfile=epub_file);

In [8]:
os.remove(txt_file)